In [47]:
import pandas as pd
import numpy as np
#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer


In [48]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [49]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [50]:
train.shape

(16278, 18)

In [51]:
val.shape

(5426, 18)

In [52]:
test.shape

(5427, 18)

In [53]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 'Yes', 'No')
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 











In [54]:
train = clean(train)
val = clean(val)
test = clean(test)


In [55]:
train['Raised Series A'].value_counts()

No     12300
Yes      188
Name: Raised Series A, dtype: int64

In [56]:
train.shape , val.shape, test.shape

((12488, 19), (4557, 19), (4552, 19))

In [57]:
train['Investor Names'].isnull().sum()

3587

In [63]:
def split_columns(text):
    text = text.copy()
    # new data frame with split value columns 
    new = text["Organization Industries"].str.split(",", n = 7, expand = True) 
    # making separate first name column from new data frame 
    text["Primary Industry"]= new[0] 
    # making separate last name column from new data frame 
    text["Sub_Ind"]= new[1] 
    text["Sub_Ind2"]= new[2]
    text["Sub_Ind3"]= new[3]
    text["Sub_Ind4"]= new[4]
    text["Sub_Ind5"]= new[5]
    text["Sub_Ind6"]= new[6]
    text["Sub_Ind7"]= new[7]
    # Dropping old Name columns 
    text.drop(columns =["Organization Industries"], inplace = True)

    #same thing but for investor column 
    new1 = text["Investor Names"].str.split(",", n = 11, expand = True) 
    # making separate first name column from new data frame 
    text["Investor1"]= new1[0] 
    # making separate last name column from new data frame 
    text["Investor1"]= new1[1] 
    text["Investor2"]= new1[2]
    text["Investor3"]= new1[3]
    text["Investor4"]= new1[4]
    text["Investor5"]= new1[5]
    text["Investor6"]= new1[6]
    text["Investor7"]= new1[7]
    text["Investor8"]= new1[8]
    text["Investor9"]= new1[9]
    text["Investor10"]= new1[10]
    text["Investor11"]= new1[11]
    # Dropping old Name columns 
    text.drop(columns =["Investor Names"], inplace = True)
    text.drop(columns =["Lead Investors"], inplace = True)



    return text 



In [64]:
train = split_columns(train)
val = split_columns(val)
test = split_columns(test)


In [65]:
train.columns

Index(['Transaction Name', 'Transaction Name URL', 'Organization Name',
       'Organization Name URL', 'Funding Type', 'Money Raised',
       'Money Raised Currency', 'Money Raised Currency (in USD)',
       'Announced Date', 'Total Funding Amount',
       'Total Funding Amount Currency',
       'Total Funding Amount Currency (in USD)', 'Organization Description',
       'Organization Location', 'Number of Investors', 'Raised Series A',
       'Primary Industry', 'Sub_Ind', 'Sub_Ind2', 'Sub_Ind3', 'Sub_Ind4',
       'Sub_Ind5', 'Sub_Ind6', 'Sub_Ind7', 'Investor1', 'Investor2',
       'Investor3', 'Investor4', 'Investor5', 'Investor6', 'Investor7',
       'Investor8', 'Investor9', 'Investor10', 'Investor11'],
      dtype='object')

In [66]:
train.isnull().sum()

Transaction Name                              0
Transaction Name URL                          0
Organization Name                             0
Organization Name URL                         0
Funding Type                                  0
Money Raised                               4176
Money Raised Currency                      4174
Money Raised Currency (in USD)             4177
Announced Date                                1
Total Funding Amount                       3091
Total Funding Amount Currency              3089
Total Funding Amount Currency (in USD)     3092
Organization Description                      0
Organization Location                       302
Number of Investors                        3587
Raised Series A                               0
Primary Industry                            490
Sub_Ind                                    2223
Sub_Ind2                                   4553
Sub_Ind3                                   7674
Sub_Ind4                                

In [62]:
#def nlp_clean(z):
   # z['Organization Description'] = z['Organization Description'].apply(lambda x: x.lower())
   # z['Organization Industries'] = z['Organization Industries'].str.lower()
   # z['Organization Location'].astype(str)
   # z['Organization Location'] = z['Organization Location'].str.lower()
    #z['Investor Names'] = z['Investor Names'].str.lower()
    #z['Lead Investors'] = z['Lead Investors'].str.lower()
    #return z 
